参考
https://jishuin.proginn.com/p/763bfbd700c2



In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("/Users/renzhiqiang/Workspace/data/iops-kpi/train.csv")
kpi=df[df["KPI ID"]=='02e99bd4f6cfb33f']
kpi.head()

,timestamp,value,label,KPI ID
0,1493568000,1.901639,0,02e99bd4f6cfb33f
1,1493568060,1.786885,0,02e99bd4f6cfb33f
2,1493568120,2.000000,0,02e99bd4f6cfb33f
3,1493568180,1.885246,0,02e99bd4f6cfb33f
4,1493568240,1.819672,0,02e99bd4f6cfb33f


In [3]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(kpi['value'].values.reshape(-1, 1))
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)

In [4]:
train_data_normalized

tensor([-0.6684, -0.6998, -0.6415,  ..., -0.4735, -0.4399, -0.5071])

In [75]:
from sklearn.preprocessing import MinMaxScaler

def get_train_data(data, time_step=10):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    normalized_data = scaler.fit_transform(data.reshape(-1, 1))
    print(np.shape(normalized_data))
    
    train_x, train_y = [], []
    
    for i in range(len(normalized_data) - time_step - 1):
        train_x.append(normalized_data[i:(i+time_step)])
        train_y.append(normalized_data[i+time_step])
    
    return np.array(train_x), np.array(train_y)

trainX, trainY = get_train_data(kpi['value'].values)

(128562, 1)


In [76]:
trainX.shape, trainY.shape

((128551, 10, 1), (128551, 1))

In [88]:
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = X
        self.y = y
           
    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X) 

data_train = TimeSeriesDataset(trainX, trainY)
data_loader_train = DataLoader(data_train, batch_size=64, pin_memory=True, shuffle=False)

In [143]:
class LSTMPredict(nn.Module):
    def __init__(self, input_size, hidden_size, seq_len=10, layer_num=2):
        super(LSTMPredict, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=layer_num, dropout=0.5)
        self.hidden_out = nn.Linear(hidden_size, 1)
        
        self.h_s = None
        self.h_c = None
        
        self.seq_len = seq_len
        self.hidden_size = hidden_size
        self.layer_num = layer_num
        
    def reset_hidden_state(self):
        self.hidden=(
            torch.zeros(self.layer_num, self.seq_len, self.hidden_size),
            torch.zeros(self.layer_num, self.seq_len, self.hidden_size)
        )
        
    def forward(self, x):
        r_out, (h_s, h_c) = self.lstm(x, self.hidden)
        output = self.hidden_out(r_out.view(self.seq_len, len(x), self.hidden_size)[-1])
        return output
        

In [144]:
model = LSTMPredict(1, 32)
loss_function = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [145]:
x, y = next(iter(data_loader_train))

In [147]:
epochs = 3

for i in range(epochs):
    total_train_loss = []
    model.train()
    for step, (b_x, b_y) in enumerate(data_loader_train):
        
        #print(b_x.shape, b_x.size(-1))
        #print(type(b_x))
        model.reset_hidden_state()
        prediction = model(b_x.to(torch.float32))
        
        #print(prediction.shape, b_y.shape)
        optimizer.zero_grad()
        single_loss = loss_function(prediction.view(-1, 1), b_y.to(torch.float32))
        total_train_loss.append(single_loss)
        single_loss.backward()
        optimizer.step()
        #print(step)
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
    

epoch:   0 loss: 0.15283582
epoch:   1 loss: 0.14977801
epoch:   2 loss: 0.14138746
epoch:   3 loss: 0.11896440
epoch:   4 loss: 0.12235024
epoch:   5 loss: 0.12062144
epoch:   6 loss: 0.12202520
epoch:   7 loss: 0.18647559
epoch:   8 loss: 0.14313266
epoch:   9 loss: 0.19298013
epoch:  10 loss: 0.13311486
epoch:  11 loss: 0.17308266
epoch:  12 loss: 0.11983070
epoch:  13 loss: 0.20174898
epoch:  14 loss: 0.18559262
epoch:  15 loss: 0.23120174
epoch:  16 loss: 0.15475635
epoch:  17 loss: 0.12831305
epoch:  18 loss: 0.18714251
epoch:  19 loss: 0.20672846
epoch:  20 loss: 0.19020726
epoch:  21 loss: 0.16894551
epoch:  22 loss: 0.18799607
epoch:  23 loss: 0.15730071
epoch:  24 loss: 0.16167255
epoch:  25 loss: 0.12605976
epoch:  26 loss: 0.17143360
epoch:  27 loss: 0.15351622
epoch:  28 loss: 0.13084440
epoch:  29 loss: 0.19345705


In [121]:
rnn = nn.LSTM(1, 32, 2)
input = torch.randn(128, 10, 1)
h0 = torch.randn(2, 10, 32)
c0 = torch.randn(2, 10, 32)
output, (hn, cn) = rnn(input, (h0, c0))

In [122]:
output.shape

torch.Size([128, 10, 32])

In [110]:
output[:, -1, :].shape

torch.Size([20, 32])

In [115]:
torch.unsqueeze(output[:, -1, :], 1).shape

torch.Size([20, 1, 32])